# Bulk Non-Managed Database Migration

1. Initialization
2. Catalog Scan
3. Bulk Migration

# Initialization
Run all cells in this initialization section each time we re attach to the cluster

In [ ]:
# Change this line to a %run if you're not using DB Repos to import the code
from DbInventoryCollector import InventoryCollector

In [ ]:
#Create widgets
InventoryCollector.CreateWidgets(dbutils, spark, reset=False)

#Initialize InventoryCollector
collector = InventoryCollector(spark, dbutils.widgets.get("Inventory_Catalog"), dbutils.widgets.get("Inventory_Database"))
collector.initialize()

sourceCatalog = dbutils.widgets.get("Scan_Catalog")
destCatalog = dbutils.widgets.get("Migration_Catalog")

# Inventory Collection


## Auto Scan All Databases
Automatically list and scan all databases.

**Parameters:**
*rescan* -- If true, will re-scan a database even if inventory data already exists for it. If false, duplicate databases will be skipped. Default: False

In [ ]:
collector.scan_all_databases(sourceCatalog, rescan = False)

## Summary of all databases

In [ ]:
database_summary = collector.get_database_inventory_summary(sourceCatalog)
display(database_summary)

# Migration Execution

## Filter database list to non-managed

In [ ]:
safeDatabases = [s.database for s in database_summary.filter("MANAGED=0 and ERROR = 0").collect()]
print(f"{len(safeDatabases)} have 0 managed tables and 0 errors")
print(safeDatabases)

## Generate and Execute all Migration DDL

In [ ]:
if destCatalog == "ChangeMeToDest" or destCatalog is None:
    raise Exception("Migration_Catalog not set. Set the destination catalog using the widgets at the top of the screen")

database_summary = collector.get_database_inventory_summary()
safeDatabases = [s.database for s in database_summary.filter("MANAGED=0 and ERROR = 0").collect()]
destCatalog = dbutils.widgets.get("Migration_Catalog")

for db in safeDatabases:
    (ddl_objects, ddl_grants) = collector.generate_migration_ddl(sourceCatalog, db, destCatalog)

    print("Finished Generation of both object and grant DDL")
    # print(';\n\n'.join(ddl_objects))
    # print(';\n\n'.join(ddl_grants))

    print(f"\n\n****\nStarting Execution for db {db}\n***")
    collector.execute_sql_list(ddl_objects)
    collector.execute_sql_list(ddl_grants)